## Задача трех тел (three-body problem)

https://ru.wikipedia.org/wiki/Задача_трёх_тел

In [ ]:
import numpy as np
from scipy.integrate import ode
import matplotlib.pyplot as plt
from numba import njit

#### Правая часть системы ОДУ (ODE right part)

In [ ]:
def right_part(t, s, constants):
    p1 = s[0:3]
    v1 = s[3:6]
    p2 = s[6:9]
    v2 = s[9:12]
    p3 = s[12:15]
    v3 = s[15:18]
    gm1, gm2, gm3 = constants

    r12 = np.linalg.norm(p2 - p1)**3
    r13 = np.linalg.norm(p3 - p1)**3
    r23 = np.linalg.norm(p3 - p2)**3
    
    # ODE
    dp1 = v1
    dp2 = v2
    dp3 = v3
    dv1 = gm2 * (p2 - p1) / r12 + gm3 * (p3 - p1) / r13
    dv2 = gm1 * (p1 - p2) / r12 + gm3 * (p3 - p2) / r23
    dv3 = gm1 * (p1 - p3) / r13 + gm2 * (p2 - p3) / r23
    
    ds = np.empty(18)
    ds[0:3] = dp1
    ds[3:6] = dv1
    ds[6:9] = dp2
    ds[9:12] = dv2
    ds[12:15] = dp3
    ds[15:18] = dv3
    
    return ds

#### Начальное состояние (initial state)

In [ ]:
# равносторонний треугольник (equal sides triangle)

angles = np.deg2rad([0, 120, 240])
r = 10
x = r * np.cos(angles)
y = r * np.sin(angles)
plt.triplot(x, y)
plt.plot(x, y, '.k')
for i in range(3):
    plt.text(x[i], y[i], f' {i+1}')
plt.axis('equal');

In [ ]:
# небольшая асимметрия в скоростях (small velocity asymmetry)

s0 = np.array([x[0], y[0], 0.0, 0.0, 0., 0.,   # положение и скорость тела 1
               x[1], y[1], 0.0, 0.5, 0., 0.,   # положение и скорость тела 2
               x[2], y[2], 0.0, 0.0, 0., 0.,]) # положение и скорость тела 3

#### Гравитационные параметры тел (gravitaional parameters)

In [ ]:
m = np.array([1., 1., 1.])*1e1

#### Интегратор Рунге-Кутты 8 порядка (Runge-Kutta integrator of 8 order)

In [ ]:
integrator = ode(njit(right_part).compile('f8[:](f8,f8[:],f8[:])'))
integrator.set_integrator('dop853')
integrator.set_f_params(m)

#### Интегрирование равными шагами (integration using equal steps)

In [ ]:
integrator.set_initial_value(s0, 0.0)
dt = 0.01
maxt = 30
n = int(maxt / dt)
states = np.empty((n, 18))
time = np.empty(n)

for i in range(n):
    time[i] = dt * (i + 1)
    states[i] = integrator.integrate(time[i])

#### Траектории в плоскости X-Y (trajectories in X-Y plane)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(*states[:,[0,1]].T, label='1');
plt.plot(*states[:,[6,7]].T, label='2');
plt.plot(*states[:,[12,13]].T, label='3');
x = s0[0::6]
y = s0[1::6]
vx = s0[3::6]
vy = s0[4::6]
plt.plot(x, y, '.k', label='start')
plt.quiver(x, y, vx, vy, scale=10, width=0.005, alpha=0.5)
for i in range(3):
    plt.text(x[i], y[i], f' {i+1}')
plt.legend()
plt.xlabel('x')
plt.ylabel('y');

#### Фазовые портреты VX-X (phase trajectories in VX-X plane)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(*states[:,[0,3]].T, label='1');
plt.plot(*states[:,[6,9]].T, label='2');
plt.plot(*states[:,[12,15]].T, label='3');
x = s0[0::6]
vx = s0[3::6]
plt.plot(x, vx, '.k', label='start')
for i in range(3):
    plt.text(x[i], vx[i], f' {i+1}')
plt.legend()
plt.xlabel('x')
plt.ylabel('vx');

#### Частное решение "восьмерка" (particular solution "number eight")

https://arxiv.org/pdf/math/0011268.pdf

In [ ]:
x1 = 0.97000436
y1 = -0.24308753
x2 = -x1
y2 = -y1
x3 = 0
y3 = 0
vx3 = -0.93240737
vy3 = -0.86473146
vx1 = -0.5*vx3
vy1 = -0.5*vy3
vx2 = vx1
vy2 = vy1

In [ ]:
m = np.array([1., 1., 1.])
integrator.set_f_params(m)

In [ ]:
s0 = np.array([x1, y1, 0.0, vx1, vy1, 0.0,   # положение и скорость тела 1
               x2, y2, 0.0, vx2, vy2, 0.0,   # положение и скорость тела 2
               x3, y3, 0.0, vx3, vy3, 0.0,]) # положение и скорость тела 3

In [ ]:
integrator.set_initial_value(s0, 0.0)
n = 1000
maxt = 6.32591398
dt = maxt / n
states = np.empty((n, 18))
time = np.empty(n)

for i in range(n):
    time[i] = dt * (i + 1)
    states[i] = integrator.integrate(time[i])

#### Траектории в плоскости X-Y (trajectories in X-Y plane)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(*states[:,[0,1]].T, label='1');
plt.plot(*states[:,[6,7]].T, label='2');
plt.plot(*states[:,[12,13]].T, label='3');
x = s0[0::6]
y = s0[1::6]
vx = s0[3::6]
vy = s0[4::6]
plt.plot(x, y, '.k', label='start')
plt.quiver(x, y, vx, vy, scale=10, width=0.005, alpha=0.5)
for i in range(3):
    plt.text(x[i], y[i], f' {i+1}')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.axis('equal');

#### Фазовые портреты VX-X (phase trajectories in VX-X plane)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(*states[:,[0,3]].T, label='1');
plt.plot(*states[:,[6,9]].T, label='2');
plt.plot(*states[:,[12,15]].T, label='3');
x = s0[0::6]
vx = s0[3::6]
plt.plot(x, vx, '.k', label='start')
for i in range(3):
    plt.text(x[i], vx[i], f' {i+1}')
plt.legend()
plt.xlabel('x')
plt.ylabel('vx');